In [1]:
import os
import re
import sys
import yaml
import optuna
import joblib
import copy
import subprocess
import shutil
from tdc.single_pred import ADME
from sklearn.preprocessing import MinMaxScaler
from optuna.storages import JournalFileStorage, JournalStorage
import pandas as pd
import numpy as np
from rdkit import Chem
from rdkit.Chem import AllChem
from ase import Atoms
from ase.io import read, write
from ase.calculators.singlepoint import SinglePointCalculator
from project_resources.import_utils import NotebookFinder
sys.meta_path.append(NotebookFinder())
from project_resources.cytochrome_P450 import smiles_to_xyz_file, get_unique_elements

importing Jupyter notebook from /home/lukas/Documents/datacytochromy/project_resources/cytochrome_P450.ipynb


In [2]:
tdc_benchmarks = ["obach", "microsome", "hepatocyte"]
# sampler - a method used to generate new sets of hyperparameters in each iteration of the optimization process
samplers = {
    'RandomSampler': optuna.samplers.RandomSampler,          # Sampler that selects hyperparameters randomly from the search space.
    'GridSampler': optuna.samplers.GridSampler,              # Sampler that performs a grid search over the hyperparameter space.
    'TPESampler': optuna.samplers.TPESampler,                # Sampler that uses a tree-structured Parzen estimator to model the objective function and sample new points from the search space.
    'CmaEsSampler': optuna.samplers.CmaEsSampler,            # Sampler that uses the Covariance Matrix Adaptation Evolution Strategy algorithm to efficiently search the hyperparameter space.
    'NSGAIISampler': optuna.samplers.NSGAIISampler,          # Multi-objective evolutionary algorithm that generates new samples using non-dominated sorting and crowding distance selection.
    'QMCSampler': optuna.samplers.QMCSampler,                # Quasi-Monte Carlo sampler that uses low-discrepancy sequences to sample the search space in a more efficient and evenly distributed way than random sampling.
    'BoTorchSampler': optuna.integration.BoTorchSampler,     # Sampler that leverages the BoTorch library for Bayesian optimization and can handle both continuous and categorical hyperparameters.
    'BruteForceSampler': optuna.samplers.BruteForceSampler,  # Sampler that exhaustively evaluates all possible combinations of hyperparameters in the search space.
}
# pruner - a technique used to eliminate unpromising trials during the course of hyperparameter optimization.
pruners = {
    'BasePruner': optuna.pruners.BasePruner,                            # This is the base class for all pruning strategies in Optuna. It provides a skeleton for implementing custom pruning strategies.
    'MedianPruner': optuna.pruners.MedianPruner,                        # A pruner that prunes unpromising trials that have median objective values, as determined in previous steps.
    'SuccessiveHalvingPruner': optuna.pruners.SuccessiveHalvingPruner,  # This pruner repeatedly splits trials into halves, discarding the lower performing half at each iteration.
    'HyperbandPruner': optuna.pruners.HyperbandPruner,                  # This pruner implements the Hyperband algorithm, which selects promising trials and runs them with different resource allocation schemes to determine the best one.
    'PercentilePruner': optuna.pruners.PercentilePruner,                # A pruner that prunes unpromising trials based on their percentile rank relative to all completed trials.
    'NopPruner': optuna.pruners.NopPruner,                              # A pruner that does nothing and does not prune any trials.
    'ThresholdPruner': optuna.pruners.ThresholdPruner,                  # This pruner prunes trials that have not reached a certain level of performance (i.e., objective value).
    'PatientPruner': optuna.pruners.PatientPruner,                      # This pruner prunes trials that do not show improvement over a certain number of steps (or epochs).
}
tdc_datasets = {}
unique_symbols = {}

In [3]:
obach = ADME(name='Half_Life_Obach')
obach_split = obach.get_split()
tdc_datasets["obach"] = obach_split
microsome = ADME(name='Clearance_Microsome_AZ')
microsome_split = microsome.get_split()
tdc_datasets["microsome"] = microsome_split
hepatocyte = ADME(name='Clearance_Hepatocyte_AZ')
hepatocyte_split = hepatocyte.get_split()
tdc_datasets["hepatocyte"] = hepatocyte_split

Found local copy...
Loading...
Done!
Found local copy...
Loading...
Done!
Found local copy...
Loading...
Done!


In [4]:
for benchmark in tdc_benchmarks:
    benchmark_train_smiles = tdc_datasets[benchmark]["train"]["Drug"]
    benchmark_test_smiles = tdc_datasets[benchmark]["test"]["Drug"]
    
    benchmark_train_halflives = tdc_datasets[benchmark]["train"]["Y"]
    reshaped_train_halflife = np.array(benchmark_train_halflives).reshape(-1, 1)
    scaler = MinMaxScaler().fit(reshaped_train_halflife)
    train_halflife_scaled = scaler.transform(reshaped_train_halflife)
    train_halflives_scaled = np.array([val[0] for val in train_halflife_scaled])

    benchmark_test_halflives = tdc_datasets[benchmark]["test"]["Y"]
    reshaped_test_halflife = np.array(benchmark_test_halflives).reshape(-1, 1)
    scaler = MinMaxScaler().fit(reshaped_test_halflife)
    test_halflife_scaled = scaler.transform(reshaped_test_halflife)
    test_halflives_scaled = np.array([val[0] for val in test_halflife_scaled])
    
    file_location = "project_resources/nequip/positions"

    smiles_to_xyz_file(benchmark_train_smiles, train_halflives_scaled, f"{file_location}/{benchmark}_train.extxyz")
    smiles_to_xyz_file(benchmark_test_smiles, test_halflives_scaled, f"{file_location}/{benchmark}_test.extxyz")

obach_train.extxyz already exists
obach_test.extxyz already exists
microsome_train.extxyz already exists
microsome_test.extxyz already exists
hepatocyte_train.extxyz already exists
hepatocyte_test.extxyz already exists


In [5]:
for benchmark in tdc_benchmarks:
    benchmark_train_smiles = tdc_datasets[benchmark]["train"]["Drug"]
    benchmark_test_smiles = tdc_datasets[benchmark]["test"]["Drug"]
    benchmark_all_smiles = list(benchmark_train_smiles) + list(benchmark_test_smiles)
    
    unique_symbs = get_unique_elements(benchmark_all_smiles)
    unique_symbols[benchmark] = list(unique_symbs)
    print(f"chemical elements present in {benchmark}: {unique_symbols[benchmark]}")

chemical elements present in obach: ['N', 'H', 'I', 'Li', 'O', 'P', 'C', 'Na', 'Br', 'Cl', 'F', 'S', 'B']
chemical elements present in microsome: ['N', 'H', 'I', 'O', 'P', 'C', 'Br', 'Cl', 'F', 'S', 'B']
chemical elements present in hepatocyte: ['N', 'H', 'I', 'O', 'P', 'C', 'Br', 'Cl', 'F', 'S', 'B']


In [6]:
def create_nequip_eval_yaml(yaml_path, root, trial_number, eval_dataset_file_name, unique_symbols):
    with open(yaml_path, "w") as out_f:
        data = {}
        
        # root is the same as nequip-train, except at the end is also included the number of the run
        data["root"] = root
        data["chemical_symbols"] = unique_symbols
        data["dataset_file_name"] = eval_dataset_file_name
        data["dataset"] = "ase"
        
        yaml.dump(data, out_f)
        benchmark = root.split("/")[-1]
        print(f"""The evaluation yaml file for the {trial_number}. iteration of {benchmark}
        with {root} as root and was successfully created """)
        out_f.close()

In [7]:
def create_nequip_train_yaml(yaml_path, batch_size, learning_rate, num_layers, root, trial_number,
                       train_dataset_file_name, unique_symbols, num_mols):
    with open(yaml_path, "w") as out_f:
        data = {}

        data["batch_size"] = batch_size
        data["learning_rate"] = learning_rate
        data["num_layers"] = num_layers

        data["root"] = root  # project_resources/nequip/{dataset}
        data["run_name"] = str(trial_number)  # number of the trial for a given combination of splitter and isozyme
        data["dataset_file_name"] = train_dataset_file_name
        data["seed"] = 123
        data["dataset_seed"] = 456
        data["append"] = True
        data["model_builders"] = ["SimpleIrrepsConfig", "EnergyModel", "PerSpeciesRescale", "RescaleEnergyEtc"]

        data["max_epochs"] = 3
        data["r_max"] = 0.4  # cutoff radius in Angstroms; changing the value has no effect on validation_e_mae
        data["l_max"] = 2
        data["parity"] = False  # slighly worse performance than True, but training takes way less time
        data["num_features"] = 32
        data["num_basis"] = 8

        data["dataset"] = "ase"
        key_mapping = {"z": "atomic_numbers", "E": "total_energy", "R": "pos"}
        data["key_mapping"] = key_mapping
        data["npz_fixed_field_keys"] = "atomic_numbers"
        data["chemical_symbols"] = unique_symbols
        data["wandb"] = False  # impossible to use wandb inside a notebook (cannot choose an option)

        num_train_val = num_mols  # number of molecules in the dataset
        num_train = int(np.floor(0.8 * num_train_val))
        num_validation = int(num_train_val - num_train)
        data["n_train"] = num_train
        data["n_val"] = num_validation
        data["train_val_split"] = "sequential"
        data["validation_batch_size"] = num_validation
        data["loss_coeffs"] = "total_energy"
        data["optimizer_name"] = "Adam"

        benchmark = root.split("/")[-1]
        
        yaml.dump(data, out_f)
        print(f"""The train yaml file for the {trial_number}. iteration of {benchmark}
        with {root} as root and was successfully created """)
        out_f.close()

In [8]:
def parse_nequip_xyz_out(xyz):
    # get the predicted half-lives from xyz file created after using nequip-evaluate --output
    with open(xyz) as f:
        energies = [float(re.search(r'energy=(-?\d+\.\d+)', line).group(1)) for line in f if "energy" in line]
    return energies

In [9]:
def parse_nequip_log(log):
    # get the rmse value from the log file created after using nequip-evaluate --log
    with open(log) as l:
        file_content = l.read()
    # use regular expressions to find the rmse value
    rmse = re.search(r'\s*e_rmse\s*=\s*(\d+\.\d+)', file_content)
    return float(rmse.group(1))

In [10]:
class NequIPTuner():
    def __init__(self, run_name, unique_symbols, num_mols, train_dataset_file_name, eval_dataset_file_name):
        self.run_name = run_name
        self.root = root
        self.unique_symbols = unique_symbols
        self.num_mols = num_mols  # number of mols present in the train .extxyz file
        self.train_dataset_file_name = train_dataset_file_name
        self.eval_dataset_file_name = eval_dataset_file_name

    def sample_params(self, trial: optuna.Trial):
        batch_size = trial.suggest_int("batch_size", 2, 50)
        learning_rate = trial.suggest_float("learning_rate", 0.001, 0.1)
        num_layers = trial.suggest_int("num_layers", 3, 5)
        return {
            "batch_size": batch_size,
            "learning_rate": learning_rate,
            "num_layers": num_layers
        }

    def train_test_return(self, train_yaml_name, eval_yaml_name, train_dir, return_predictions=False):
        xyz_out = f"{train_dir}/output.xyz"
        log_file = f"{train_dir}/evaluation_log.txt"
        
        print(f"running command: nequip-train {train_yaml_name}")
        !nequip-train {train_yaml_name}
        # {train_dir} ... project_resources/optuna/nequip/random/3A4/1
        # {eval_yaml_name} ... eval_dataset.yaml
        # {xyz_out} ... project_resources/optuna/nequip/random/3A4/1/output.xyz
        # ^ Parse this to get the predicted half-lives ^
        # {log} ... project_resources/optuna/nequip/random/3A4/1/eval_log.txt
        
        print(f"""running command: nequip-evaluate --train-dir {train_dir} 
              --dataset-config {eval_yaml_name} 
              --output {xyz_out} 
              --log {log_file}""")
        !nequip-evaluate --train-dir {train_dir} --dataset-config {eval_yaml_name} --output {xyz_out} --log {log_file}

        rmse = parse_nequip_log(log_file)
        
        # delete the dirs and files that were created, as they are no longer needed
        shutil.rmtree(train_dir)
        os.remove(train_yaml_name)
        os.remove(eval_yaml_name)
        
        if return_predictions:
            predictions = parse_nequip_xyz_out(xyz_out)
            return rmse, predictions
        else:
            return rmse

    def objective(self, trial=None):
        parameters = self.sample_params(trial)
        root = f"project_resources/optuna/nequip/{self.run_name}"
        train_yaml_name = f"{root}/nequip_train_{trial.number}.yaml"
        eval_yaml_name = f"{root}/nequip_eval_{trial.number}.yaml"
        train_dir = root + f"/{trial.number}"
        
        # create train yaml file for this trial
        create_nequip_train_yaml(train_yaml_name, parameters["batch_size"], parameters["learning_rate"],
                           parameters["num_layers"], root, trial.number, self.train_dataset_file_name,
                           self.unique_symbols, self.num_mols)

        # create evaluation yaml file for this trial
        create_nequip_eval_yaml(eval_yaml_name, root, trial.number,
                                self.eval_dataset_file_name, self.unique_symbols)

        return self.train_test_return(train_yaml_name, eval_yaml_name, train_dir)

In [11]:
sampler = samplers["TPESampler"]
pruner = pruners["BasePruner"]
n_trials = 1
for benchmark in tdc_benchmarks:
    print(benchmark)
    root = f"project_resources/optuna/nequip/{benchmark}"
    lock_obj = optuna.storages.JournalFileOpenLock(root + "/nequip_journal.log")
    storage = JournalStorage(
        JournalFileStorage(root + "/nequip_journal.log", lock_obj=lock_obj)
    )

    study = optuna.create_study(study_name=f"nequip_{benchmark}", directions=["minimize"],
                                pruner=pruner, storage=storage, load_if_exists=True)

    run_name = benchmark
    
    num_mols = len(tdc_datasets[benchmark]["train"]["Drug"])
    train_dataset_file_name = f"project_resources/nequip/positions/{benchmark}_train.extxyz"
    eval_dataset_file_name = f"project_resources/nequip/positions/{benchmark}_test.extxyz"
    tuner = NequIPTuner(run_name, unique_symbols[benchmark], num_mols, train_dataset_file_name, eval_dataset_file_name)

    study.optimize(tuner.objective, n_trials=n_trials, n_jobs=-1)
    joblib.dump(study, f"{root}/nequip.pkl")

[I 2023-12-04 19:18:21,425] A new study created in Journal with name: nequip_obach


obach
The train yaml file for the 0. iteration of obach
        with project_resources/optuna/nequip/obach as root and was successfully created 
The evaluation yaml file for the 0. iteration of obach
        with project_resources/optuna/nequip/obach as root and was successfully created 
running command: nequip-train project_resources/optuna/nequip/obach/nequip_train_0.yaml
Torch device: cpu
Processing dataset...
multiprocessing.pool.RemoteTraceback: 
"""
Traceback (most recent call last):
  File "/home/lukas/anaconda3/envs/soc/lib/python3.7/multiprocessing/pool.py", line 121, in worker
    result = (True, func(*args, **kwds))
  File "/home/lukas/anaconda3/envs/soc/lib/python3.7/multiprocessing/pool.py", line 44, in mapstar
    return list(map(*args))
  File "/home/lukas/anaconda3/envs/soc/lib/python3.7/site-packages/nequip/data/dataset.py", line 790, in _ase_dataset_reader
    if global_index in include_frames
  File "/home/lukas/anaconda3/envs/soc/lib/python3.7/site-packages/nequip/d

[W 2023-12-04 19:19:08,993] Trial 0 failed with parameters: {'batch_size': 28, 'learning_rate': 0.0462399708745702, 'num_layers': 3} because of the following error: FileNotFoundError(2, 'No such file or directory').
Traceback (most recent call last):
  File "/home/lukas/anaconda3/envs/soc/lib/python3.7/site-packages/optuna/study/_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
  File "/tmp/ipykernel_27390/992946843.py", line 67, in objective
    return self.train_test_return(train_yaml_name, eval_yaml_name, train_dir)
  File "/tmp/ipykernel_27390/992946843.py", line 38, in train_test_return
    rmse = parse_nequip_log(log_file)
  File "/tmp/ipykernel_27390/3300226570.py", line 3, in parse_nequip_log
    with open(log) as l:
FileNotFoundError: [Errno 2] No such file or directory: 'project_resources/optuna/nequip/obach/0/evaluation_log.txt'
[W 2023-12-04 19:19:08,994] Trial 0 failed with value None.
[I 2023-12-04 19:19:09,001] A new study created in Journal with n

microsome
The train yaml file for the 0. iteration of microsome
        with project_resources/optuna/nequip/microsome as root and was successfully created 
The evaluation yaml file for the 0. iteration of microsome
        with project_resources/optuna/nequip/microsome as root and was successfully created 
running command: nequip-train project_resources/optuna/nequip/microsome/nequip_train_0.yaml
Torch device: cpu
Processing dataset...
Loaded data: Batch(atomic_numbers=[22572, 1], batch=[22572], cell=[772, 3, 3], edge_cell_shift=[182674, 3], edge_index=[2, 182674], pbc=[772, 3], pos=[22572, 3], ptr=[773], total_energy=[772, 1])
    processed data size: ~5.52 MB
Cached processed data to disk
Done!
Successfully loaded the data set of type ASEDataset(772)...
Replace string dataset_per_atom_total_energy_std to 0.0097426138818264
Replace string dataset_per_atom_total_energy_mean to 0.0061898925341665745
Atomic outputs are scaled by: [H, B, C, N, O, F, P, S, Cl, Br, I: 0.009743], shifted by

[I 2023-12-04 19:22:35,908] Trial 0 finished with value: 0.306079 and parameters: {'batch_size': 45, 'learning_rate': 0.09820617685184492, 'num_layers': 5}. Best is trial 0 with value: 0.306079.
[I 2023-12-04 19:22:35,915] A new study created in Journal with name: nequip_hepatocyte


hepatocyte
The train yaml file for the 0. iteration of hepatocyte
        with project_resources/optuna/nequip/hepatocyte as root and was successfully created 
The evaluation yaml file for the 0. iteration of hepatocyte
        with project_resources/optuna/nequip/hepatocyte as root and was successfully created 
running command: nequip-train project_resources/optuna/nequip/hepatocyte/nequip_train_0.yaml
Torch device: cpu
Processing dataset...
Loaded data: Batch(atomic_numbers=[24917, 1], batch=[24917], cell=[849, 3, 3], edge_cell_shift=[208448, 3], edge_index=[2, 208448], pbc=[849, 3], pos=[24917, 3], ptr=[850], total_energy=[849, 1])
    processed data size: ~6.27 MB
Cached processed data to disk
Done!
Successfully loaded the data set of type ASEDataset(849)...
Replace string dataset_per_atom_total_energy_std to 0.013990852050483227
Replace string dataset_per_atom_total_energy_mean to 0.009903610683977604
Atomic outputs are scaled by: [H, B, C, N, O, F, P, S, Cl, Br, I: 0.013991], shi

[W 2023-12-04 19:25:36,854] Trial 0 failed with parameters: {'batch_size': 23, 'learning_rate': 0.02506898471129209, 'num_layers': 4} because of the following error: AttributeError("'NoneType' object has no attribute 'group'").
Traceback (most recent call last):
  File "/home/lukas/anaconda3/envs/soc/lib/python3.7/site-packages/optuna/study/_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
  File "/tmp/ipykernel_27390/992946843.py", line 67, in objective
    return self.train_test_return(train_yaml_name, eval_yaml_name, train_dir)
  File "/tmp/ipykernel_27390/992946843.py", line 38, in train_test_return
    rmse = parse_nequip_log(log_file)
  File "/tmp/ipykernel_27390/3300226570.py", line 7, in parse_nequip_log
    return float(rmse.group(1))
AttributeError: 'NoneType' object has no attribute 'group'
[W 2023-12-04 19:25:36,855] Trial 0 failed with value None.
